In [ ]:
%reload_ext nb_black

In [ ]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from scipy import stats

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def plot_guess(
    data,
    x1_mean,
    x1_std,
    x2_mean,
    x2_std,
    n=int(1e5),
    prob_1=None,
    prob_2=None,
    percent_1=0.5,
):
    """Helper function for plotting GMM process

    Parameters
    ----------
    data:      1d array of data values
    x1_mean:   the estimated mean of sample 1
    x1_std:    the estimated standard deviation of
               sample 1
    x2_mean:   the estimated mean of sample 2
    x2_std:    the estimated standard deviation of
               sample 2
    n:         the number of samples to generate for x1 &
               x2 plotting samples
    prob_1:    1d array of the probability associated with each
               observation belonging to group 1
    prob_2:    1d array of the probability associated with each
               observation belonging to group 2
    percent_1: what percentage of data is expected
               to be in group 1 (used to control height
               of distribution)
    """
    n1 = int(n * percent_1)
    n2 = int(n * (1 - percent_1))
    sample_1 = np.random.normal(x1_mean, x1_std, n1)
    sample_2 = np.random.normal(x2_mean, x2_std, n2)

    sns.kdeplot(sample_1, label="Group 1")
    sns.kdeplot(sample_2, label="Group 2")

    if prob_1 is not None and prob_2 is not None:
        prob_1 = np.array(prob_1).reshape(-1, 1)
        prob_2 = np.array(prob_2).reshape(-1, 1)

        prob_1 = MinMaxScaler().fit_transform(prob_1)
        prob_2 = MinMaxScaler().fit_transform(prob_2)

        for x, p1, p2 in zip(data, prob_1, prob_2):
            if p1 > p2:
                c = "blue"
                alpha = p1[0]
            else:
                c = "orange"
                alpha = p2[0]

            plt.scatter(x, 0, alpha=alpha * 0.8, c=c)
    else:
        plt.scatter(data, [0 for _ in data], c="black")

    plt.yticks([])
    plt.legend()

In [ ]:
lat_min = 35.71
lat_max = 36.55
lon_min = -84.55
lon_max = -82.37

# Potential extra practice:
#  * plot all the starbs locations in one color
#  * plot my location in a different color
adam_lat = 36.3
adam_lon = -82.4

data_url = "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv"
starbs = pd.read_csv(data_url)

starbs = starbs[starbs["Latitude"].between(lat_min, lat_max)]
starbs = starbs[starbs["Longitude"].between(lon_min, lon_max)]
starbs.head(3)

## 🔥 Warm up 🔥

According to the distribution of our filtered data in the `starbs` dataframe.

We're most likely to find a starbucks at which of these values of `'Longitude'`?
* (A) -81
* (B) -83
* (C) -84
* (D) -85

According to the distribution of our filtered data in the `starbs` dataframe.

We're most likely to find a starbucks at which of these values of `'Latitude'`?
* (A) 35
* (B) 36
* (C) 37
* (D) 38

## Gaussian Mixture Model

gaussian distribution = normal distribution
 
### MLE

We're given the below data, and we're told that the data is measurements of some snail characteristic 🐌.  We want to try and figure out what the *population* distribution looks like.  Remember we just have a *sample*, but the population is all of the snail species we're studying.  Having a good estimate of this population distribution can benefit and guide our analysis.

In [ ]:
# fmt: off
data = [ 8.521, 16.586, 11.154,  3.323, 13.662, 14.649,  6.149,  5.528,
        18.871, 11.498,  8.921,  5.776,  7.292,  6.638, 13.321,  7.073,
         8.827, 10.375,  1.645, 13.566, 19.846,  6.347,  8.617, 14.462,
         4.483, 11.170, 11.322,  5.710, 11.311,  7.672,  9.765, 14.443,
        18.360,  9.304, 10.247, 10.955, 14.194,  8.344,  5.783, 12.533,
        12.937,  0.846,  4.925,  9.006, 11.443, 16.160, 10.751,  8.513,
        23.865, 17.228]

* Before plotting:
    * What kind of distribution do you expect this type of data to follow?
    * What 'parameters' does that distribution have?
        * i.e. these are what you'd need to give `np.random.<distribution_name>()` so it can know what shape of distribution you want random numbers from


* Now let's plot a histogram.  Does the shape of the data's distribution support your hypothesis?

Remember, we're after the population distribution.

* Re-plot the histogram
* Add a vertical line at 20 (color it and give it a label for the legend)
* Add a vertical line at 10 (color it and give it a label for the legend)
* Add a vertical line at  5 (color it and give it a label for the legend)

Let's say these are 3 guesses at what the population mean are.  Given our data, which of these is the most *likely*.  Due to the nature of random sampling, it's possible that all 3 of these are valid, but it's not practical to assume that we got a very unusual random sample.

We're engaging in a process called Maximum Likelihood Estimation (MLE).  We're trying to Estimate the population mean based on what's most *likely*.  We want our Estimate to have the Maximum Likelihood of being correct.

Below is a visualization of us trying to find the population mean via MLE.

Note, this is why the mean parameter of `np.random.normal` is called `loc`.

<img src='images/mean_mle.gif' width=70%>

There's one more parameter we'd need to esimate in order to fully describe our distribution's shape.  This is why this parameter in `np.random.normal` is called `scale` (we scale the width, the height is derived from the width).

<img src='images/std_mle.gif' width=60%>

In this case, the maximum likelihood estimates for the population mean and standard deviation are the equal to the sample mean and sample standard deviations.

* Generate 1000 random data points from our estimate popuation distribution
* Plot this resulting distribution, compare it to our sample

### Mixing

We just got some new data.  This data was collected from 2 separate species of snails 🐌.

We know these snails each follow a normal distribution, and we know that they have different means & standard deviations.  Unfortunately, the scientist in the field didn't write down which species each observation is, so we have to try and figure out these 2 separate distributions from a sample of mixed data.

If we knew the species labels, we could perform the same MLE process we did above (filter and perform MLE 1 at a time).

In [ ]:
# fmt: off
data = [11.83984961, 12.37143473, 26.15417807, 28.58500880, 27.70571253,
        24.24028217, 18.33611103, 15.28117383, 14.57235710, 18.49006327,
        37.83761751, 18.82148403, 36.62430095, 26.61444903, 15.3433858 ,
        24.60865873, 31.67437436, 26.08487739, 14.75279305, 25.63485726,
        30.44683604, 29.64163292, 14.91536797, 20.48912193, 27.97187397,
        11.41235662, 17.90399557, 33.82514212, 17.71352474, 25.98954934,
        19.86878159, 26.92304096, 16.25738730, 29.10667734, 31.06548273,
        21.14768063, 29.93913722, 25.32381510, 18.98788655, 16.60772929,
        25.00896332, 17.41901911, 14.21902871, 27.90108363, 26.99118323,
        26.03784060, 31.83483958, 25.73633429, 31.48278996, 24.23683382,
        19.24019041, 14.73365444, 27.70687662, 19.82397780, 14.58054905,
        22.51116415, 21.31616800, 26.34025573, 28.45094146, 24.61646750]

Plot the distribution of the data.  Do you think we'll be able to separate out 2 normal distributions from it?

To figure this out, we'll throw in some initial guesses.

In [ ]:
guess_x1_mean = ____
guess_x1_std = ____
guess_x2_mean = ____
guess_x2_std = ____

In [ ]:
# plot_guess?

In [ ]:
plot_guess(
    data,
    x1_mean=guess_x1_mean,
    x1_std=guess_x1_std,
    x2_mean=guess_x2_mean,
    x2_std=guess_x2_std,
)
plt.title("initial guess")
plt.show()

Thanks to these distributions, we can now assign a probability (or likelihood) that each point came from each distribution.  Using this, we can split the data into 2 groups:

* (1) points more likely to have come from the orange distribution
* (2) points more likely to have come from the blue distribution

In [ ]:
df = pd.DataFrame(data, columns=["mixture"])
df["prob_blue"] = stats.norm(guess_x1_mean, guess_x1_std).pdf(df["mixture"])
df["prob_orange"] = stats.norm(guess_x2_mean, guess_x2_std).pdf(df["mixture"])
df["label"] = "blue"
df.loc[df["prob_orange"] > df["prob_blue"], "label"] = "orange"
df.head(3)

In [ ]:
plot_guess(
    data,
    x1_mean=guess_x1_mean,
    x1_std=guess_x1_std,
    x2_mean=guess_x2_mean,
    x2_std=guess_x2_std,
    prob_1=df["prob_blue"],
    prob_2=df["prob_orange"],
)
plt.title("initial guess")
plt.show()

We now have 2 individual groups of data! And just like in the single species example, we can now make a better guess about what each distribution looks like.  We can use this to update our guess of the distribution shapes.

In [ ]:
descriptives = df.groupby("label").agg({"mixture": ["mean", "std"]})
descriptives.columns = ["mean", "std"]
descriptives

In [ ]:
guess_x1_mean = descriptives.loc["blue", "mean"]
guess_x1_std = descriptives.loc["blue", "std"]
guess_x2_mean = descriptives.loc["orange", "mean"]
guess_x2_std = descriptives.loc["orange", "std"]

In [ ]:
plot_guess(
    data,
    x1_mean=guess_x1_mean,
    x1_std=guess_x1_std,
    x2_mean=guess_x2_mean,
    x2_std=guess_x2_std,
    prob_1=df["prob_blue"],
    prob_2=df["prob_orange"],
)
plt.title("updated guess")
plt.show()

And repeat! This process we just went through goes by the name Gussian Mixture Modeling.  Gaussian is another name for the normal distribution (named after a dude who contributed a lot to math and statistics).

Let's see how we can approach this problem with sklearn.

In [ ]:
X = df[["mixture"]]

In [ ]:
gmm = GaussianMixture(2)
gmm.fit(X)

Here are the means and standard deviations that sklearn settled on after going through that process a couple more iterations.  Note that in this simple case, our 1 iteration algorithm got pretty similar results.

In [ ]:
x1_mean, x2_mean = gmm.means_
x1_std, x2_std = np.sqrt(gmm.covariances_)

x1_mean = x1_mean[0]
x2_mean = x2_mean[0]
x1_std = x1_std[0][0]
x2_std = x2_std[0][0]

print(f"Cluster 1 - mean: {x1_mean:.2f}; std: {x1_std:.2f}")
print(f"Cluster 2 - mean: {x2_mean:.2f}; std: {x2_std:.2f}")

group_probs = gmm.predict_proba(X)
prob_1 = group_probs[:, 0]
prob_2 = group_probs[:, 1]

plot_guess(
    data,
    x1_mean=x1_mean,
    x1_std=x1_std,
    x2_mean=x2_mean,
    x2_std=x2_std,
    prob_1=prob_1,
    prob_2=prob_2,
)
plt.title("sklearn's guess")
plt.show()

That's cool and all, but our data almost never has just a single column (i.e. this example was *univariate*).  Let's look at a case with multiple variables (i.e. *multivariate*).

In [ ]:
lat_lon = starbs[["Latitude", "Longitude"]].copy()

fig = px.scatter_geo(lat_lon, "Latitude", "Longitude", scope="usa")
fig.update_geos(fitbounds="locations")
fig.show()

We can try and visualize both distributions at once using various different methods

In [ ]:
px.scatter(
    starbs, "Longitude", "Latitude", marginal_x="histogram", marginal_y="histogram"
)

In [ ]:
x = starbs["Longitude"]
y = starbs["Latitude"]
kernel = stats.gaussian_kde((x, y))

x = np.linspace(x.min(), x.max(), 100)
y = np.linspace(y.min(), y.max(), 100)

z = []
for xi in x:
    zi = []
    for yi in y:
        zi.append(kernel((xi, yi))[0])

    z.append(zi)

z = np.array(z)
z.reshape((x.shape[0], y.shape[0]))

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.show()

Just like KMeans, we need to decide up front how many clusters we want the clustering process to find.

In [ ]:
k = 4

* Create and fit a `GaussianMixture()` model

In [ ]:
gauss = ____

Let's compare `GaussianMixture()` to `KMeans()`.

* Create and fit a `KMeans()` model

In [ ]:
kmeans = ____

* Print the resulting centroids from each method

* Use the `GaussianMixture.predict_proba()` method.
* What are the top 3 observations we are least confident about?
* What are the top 3 observations we are most confident about?

Visualizing the results comparison:

In [ ]:
plot_df = lat_lon.copy()
plot_df["label"] = gauss.predict(lat_lon)
plot_df = plot_df.sort_values("label")

centers_df = pd.DataFrame(gauss.means_, columns=["Latitude", "Longitude"])
centers_df["label"] = "Cluster center"

plot_df = pd.concat((plot_df, centers_df), sort=False)

fig = px.scatter_geo(
    plot_df,
    "Latitude",
    "Longitude",
    color="label",
    scope="usa",
    title="Gaussian Mixture Model Results",
    hover_name=plot_df.index,
)

fig.update_geos(fitbounds="locations")
fig.show()

# ----------------------------------------------------------------------------

plot_df = lat_lon.copy()
plot_df["label"] = kmeans.labels_
plot_df = plot_df.sort_values("label")

centers_df = pd.DataFrame(kmeans.cluster_centers_, columns=["Latitude", "Longitude"])
centers_df["label"] = "Cluster center"

plot_df = pd.concat((plot_df, centers_df), sort=False)

fig = px.scatter_geo(
    plot_df,
    "Latitude",
    "Longitude",
    color="label",
    scope="usa",
    title="KMeans Results",
    hover_name=plot_df.index,
)

fig.update_geos(fitbounds="locations")
fig.show()

## Mean-shift

* Use `matplotlib` to plot the East TN Starbucks
* Label the `x` and `y` axes
* Give the plot a title

* Use `sns.kdeplot()` to show the distribution of Latitude and Longitude separately.
* How many clusters do you expect to find if clustering only on one of these variables?
* Play with the `bw_method` parameter of `sns.kdeplot()`.  How does this change how many clusters you expect?

* Use `sns.kdeplot()` to show the 2d distribution of Latitude and Longitude.
* Add the starbucks locations to the plot.
* How many clusters do you expect to find if clustering only on one of these variables?
* Play with the `bw_method` parameter of `sns.kdeplot()`.  How does this change how many clusters you expect?

A slightly different view of the same plot.

In [ ]:
bw = 0.5

x = starbs["Longitude"]
y = starbs["Latitude"]
kernel = stats.gaussian_kde((x, y), bw_method=bw)

x = np.linspace(x.min(), x.max(), 100)
y = np.linspace(y.min(), y.max(), 100)

z = []
for xi in x:
    zi = []
    for yi in y:
        zi.append(kernel((xi, yi))[0])

    z.append(zi)

z = np.array(z)
z.reshape((x.shape[0], y.shape[0]))

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.show()

* Use `MeanShift()` to cluster the observations
* The default `bandwidth` used by `MeanShift()` is calculated using `sklearn.cluster.estimate_bandwidth()` (shown below)

In [ ]:
default_bw = estimate_bandwidth(lat_lon)
print(f"Default bandwidth: {default_bw}")

In [ ]:
# Define a variable to hold the selected bw for use in plotting later
bw = ___

clst = ____

* Show the resulting cluster centers.  How many clusters were found?

* Redo the above plot colored by cluster label

#### Extra practice in case we have time:

* The nba dataset is loaded & cleaned for you below.
* Apply mean shift and interpret the clusters.
* Based on your interpretation, are these good clusters?

In [ ]:
data_url = "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv"
nba = pd.read_csv(data_url)

nba = nba[(nba["GS"] >= 20) & (nba["MP"] >= 10)]
nba = nba.dropna().reset_index(drop=True)
nba_og = nba.copy()

nba = nba[["PTS", "TRB", "TOV", "AST", "BLK", "Age"]]
nba.head(3)